# IMPORTS

In [56]:
import sqlite3
import pandas as pd
import numpy as np
import math
sqlite3.sqlite_version
%matplotlib inline

from sklearn.neighbors import NearestNeighbors
from sklearn.impute import KNNImputer

database = sqlite3.connect('database.sqlite')

# DATA CLEANING
- seřadíme záznamy podle _ProductId_

- odstraníme duplikáty, aby naše výsledky byly bez biasu (velmi podobne produkty, stejne review) - nechavame jen prvni

- díváme se na velikost _HelpfulnessNumerator_ a _HelpfulnessDenominator_


- vytvoření dictionary _users_products_, kde si pro každého uživatele ukládáme koupené produkty a číselné hodnocení, které jim dal (dictionary kvůli průměrně konstantní časové složitosti dotazu)

In [57]:
df = pd.read_sql_query("select * from Reviews limit 3000", database)
df

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...
...,...,...,...,...,...,...,...,...,...,...
2995,2996,B0009JI7O8,A3KLEQQVY1I46S,Spence,2,2,4,1344297600,Color Discrepancy,"My primary reason for ordering, was the pictur..."
2996,2997,B0009JI7O8,A1WXEIP21U01ME,zazza,4,5,5,1297900800,crispy fresh,The cookie is crispy good and flavorful. The ...
2997,2998,B0009JI7O8,A7XPFLQYVPZAT,Kim Voong,1,1,2,1334361600,they are soooo tiny!,I was expecting these cookies were going to be...
2998,2999,B0009JI7O8,A3ADANTR6G4LPB,Marjory Warrington,1,1,5,1323129600,Excellent service and product,I ordered these fortune cookies as rewards for...


In [58]:
#Deduplication of entries
df = df.drop_duplicates(subset={"UserId","ProfileName","Time","Text"}, keep='first')
df.shape

(2990, 10)

In [59]:
df = df[df.HelpfulnessNumerator<=df.HelpfulnessDenominator]

In [60]:
users_products = {}
for index, radek in df.iterrows():
    product_info = []
    user_id = radek.UserId
    product_id = radek.ProductId
    score = radek.Score
    product_info.append(product_id)
    product_info.append(score)

    if user_id not in users_products:
        users_products[user_id] = []
    users_products[user_id] += [product_info]
users_products

{'A3SGXH7AUHU8GW': [['B001E4KFG0', 5]],
 'A1D87F6ZCVE5NK': [['B00813GRG4', 1]],
 'ABXLMWJIXXAIN': [['B000LQOCH0', 4]],
 'A395BORC6FGVXV': [['B000UA0QIQ', 2]],
 'A1UQRSCLF8GW1T': [['B006K2ZZ7K', 5]],
 'ADT0SRK1MGOEU': [['B006K2ZZ7K', 4]],
 'A1SP2KVKFXXRU1': [['B006K2ZZ7K', 5]],
 'A3JRGQVEQN31IQ': [['B006K2ZZ7K', 5]],
 'A1MZYO9TZK0BBI': [['B000E7L2R4', 5]],
 'A21BT40VZCCYT4': [['B00171APVA', 5]],
 'A3HDKO7OW0QNK4': [['B0001PB9FE', 5]],
 'A2725IB4YY9JEB': [['B0009XLVG0', 5]],
 'A327PCT23YH90': [['B0009XLVG0', 1]],
 'A18ECVX2RJ7HUE': [['B001GVISJM', 4]],
 'A2MUGFV2TDQ47K': [['B001GVISJM', 5], ['B000G6RYNE', 3], ['B001CWZXIY', 5]],
 'A1CZX3CP8IKQIJ': [['B001GVISJM', 5]],
 'A3KLWF6WQ5BNYO': [['B001GVISJM', 2]],
 'AFKW14U97Z6QO': [['B001GVISJM', 5]],
 'A2A9X58G2GTBLP': [['B001GVISJM', 5]],
 'A3IV7CL2C13K2U': [['B001GVISJM', 5]],
 'A1WO0KGLPR5PV6': [['B001GVISJM', 5]],
 'AZOF9E17RGZH8': [['B001GVISJM', 5]],
 'ARYVQL4N737A1': [['B001GVISJM', 5]],
 'AJ613OLZZUG7V': [['B001GVISJM', 5]],
 'A22P2J0

In [61]:
most_popular = pd.read_sql_query("""
select ProductId, sum(Score)
from Reviews
where Score in (3, 4, 5)
group by ProductId
order by sum(Score) desc
limit 6000
""", database)

most_popular

,ProductId,sum(Score)
0,B007JFMH8M,4156
1,B003B3OOPA,2932
2,B002QWP8H0,2849
3,B002QWP89S,2849
4,B002QWHJOU,2849
...,...,...
5995,B001II46YO,67
5996,B001GVIRCU,67
5997,B001FA1SI2,67
5998,B001EQ5GX0,67


# Interaction matrix
- vytvářím tabulku _im_, kde si ukládám interakce mezi každým uživatelem

In [62]:
im = df[['UserId', 'ProductId', 'Score']] #interaction matrix
im = im.drop_duplicates(['UserId', 'ProductId'])
im = im.pivot(index='UserId', columns='ProductId')
im

Score                                              \
ProductId      B00002NCJC B00002Z754 B000084E1U B0000CGFV4 B0000DC5IY   
UserId                                                                  
A10317LUD1C1VJ        NaN        NaN        NaN        NaN        NaN   
A103EZCS9H8WW1        NaN        NaN        NaN        NaN        NaN   
A104Y49ZQ4CYJ2        NaN        NaN        NaN        NaN        NaN   
A105FP1ZT88EPL        NaN        NaN        NaN        NaN        NaN   
A107MO1RZUQ8V         NaN        NaN        NaN        NaN        NaN   
...                   ...        ...        ...        ...        ...   
AZOF9E17RGZH8         NaN        NaN        NaN        NaN        NaN   
AZS05OYE0XGNF         NaN        NaN        NaN        NaN        NaN   
AZTEQJCI6N5WL         NaN        NaN        NaN        NaN        NaN   
AZV26LP92E6WU         NaN        NaN        NaN        NaN        NaN   
AZVR7NPPEDMMN         NaN        NaN        NaN        NaN        NaN   

                                                                       ...  \
ProductId      B0000VLH8S B00015BQB6 B0001FQVCK B0001OINNQ B0001PB9FE  ...   
UserId                                                                 ...   
A10317LUD1C1VJ        NaN        NaN        NaN        NaN        NaN  ...   
A103EZCS9H8WW1        NaN        NaN        NaN        NaN        NaN  ...   
A104Y49ZQ4CYJ2        NaN        NaN        NaN        NaN        NaN  ...   
A105FP1ZT88EPL        NaN        NaN        NaN        NaN        NaN  ...   
A107MO1RZUQ8V         NaN        NaN        NaN        NaN        NaN  ...   
...                   ...        ...        ...        ...        ...  ...   
AZOF9E17RGZH8         NaN        NaN        NaN        NaN        NaN  ...   
AZS05OYE0XGNF         NaN        NaN        NaN        NaN        NaN  ...   
AZTEQJCI6N5WL         NaN        NaN        NaN        NaN        NaN  ...   
AZV26LP92E6WU         NaN        NaN        NaN        NaN        NaN  ...   
AZVR7NPPEDMMN         NaN        NaN        NaN        NaN        NaN  ...   

                                                                       \
ProductId      B0087HW5E2 B0089PI9OC B0089SPDUW B008BEGP9W B008L19ZQ0   
UserId                                                                  
A10317LUD1C1VJ        NaN        NaN        NaN        NaN        NaN   
A103EZCS9H8WW1        NaN        NaN        NaN        NaN        NaN   
A104Y49ZQ4CYJ2        NaN        NaN        NaN        NaN        NaN   
A105FP1ZT88EPL        NaN        NaN        NaN        NaN        NaN   
A107MO1RZUQ8V         NaN        NaN        NaN        NaN        NaN   
...                   ...        ...        ...        ...        ...   
AZOF9E17RGZH8         NaN        NaN        NaN        NaN        NaN   
AZS05OYE0XGNF         NaN        NaN        NaN        NaN        NaN   
AZTEQJCI6N5WL         NaN        NaN        NaN        NaN        NaN   
AZV26LP92E6WU         NaN        NaN        NaN        NaN        NaN   
AZVR7NPPEDMMN         NaN        NaN        NaN        NaN        NaN   

                                                                       
ProductId      B008MMLXEK B008YAXFWI B0093NIWVO B009HINRX8 B009UOFU20  
UserId                                                                 
A10317LUD1C1VJ        NaN        NaN        NaN        NaN        NaN  
A103EZCS9H8WW1        NaN        NaN        NaN        NaN        NaN  
A104Y49ZQ4CYJ2        NaN        NaN        NaN        NaN        NaN  
A105FP1ZT88EPL        NaN        NaN        NaN        NaN        NaN  
A107MO1RZUQ8V         NaN        NaN        NaN        NaN        NaN  
...                   ...        ...        ...        ...        ...  
AZOF9E17RGZH8         NaN        NaN        NaN        NaN        NaN  
AZS05OYE0XGNF         NaN        NaN        NaN        NaN        NaN  
AZTEQJCI6N5WL         NaN        NaN        NaN        NaN        NaN  
AZV26LP92E6WU    

In [63]:
X = im.values
print(X)

[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]


# NEAREST NEIGHBORS
- hledám uživatele, kteří si kupovali nejpodobnější produkty a byli podobně spokojení (každého uživatele si reprezentujeme jako vektor určený jeho hodnoceními pro různé produkty - potom sousedy můžeme najít jednoduše nalezením nejbližších vektorů)
- lze zvolit různé metriky na určení vzdálenosti vektorů - já pro jednoduchost používám funkci _NearestNeighbors_ z knihovny _sklearn.neighbors_, která používá euklidovskou normu
- sousedy každého uživatele si ukládám do dictionary _nearest_neighbors_ (dictionary používám kvůli dobré časové složitosti)

In [65]:
X = im.values

imp = KNNImputer(n_neighbors=5)
X = imp.fit_transform(X)

knn = NearestNeighbors(n_neighbors=5, algorithm='ball_tree').fit(X)
knn

NearestNeighbors(algorithm='ball_tree')

In [66]:
distances, indices = knn.kneighbors(X)
indices

array([[   0,  478,  341,  260,  371],
       [   1,   26,    8,    9,   21],
       [   2,    1,    8,   21,    9],
       ...,
       [ 571, 2801, 1548,  358,  765],
       [2739, 2889,  756,    1,    8],
       [ 265,  215,  272,  180,  245]])

In [67]:
distances

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 2., 2., 2., 2.],
       ...,
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 1.],
       [0., 0., 0., 0., 0.]])

In [68]:
knn_graph = knn.kneighbors_graph(X, mode='distance').toarray()
# metric='cosine', include_self=False
knn_graph

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 2., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [69]:
nearest_neighbors = {}
for i in range(knn_graph.shape[0]):
    user_id = im.index.values[i]
    x = []

    for j in range(knn_graph.shape[1]):
        if knn_graph[i][j] != 0.0:
            neighbor_id = im.index.values[j]
            x.append(neighbor_id)

    nearest_neighbors[user_id] = x

nearest_neighbors

{'A10317LUD1C1VJ': [],
 'A103EZCS9H8WW1': [],
 'A104Y49ZQ4CYJ2': ['A103EZCS9H8WW1',
  'A10EHUTGNC4BGP',
  'A10IFYN5U6X20R',
  'A11NKS291CE5VK'],
 'A105FP1ZT88EPL': [],
 'A107MO1RZUQ8V': [],
 'A107SVKYGPGBPP': [],
 'A108P30XVUFKXY': ['A26AY1TFK8BQXQ',
  'A276999Y6VRSCQ',
  'A2CI0RLADCRKPF',
  'A39Z97950MCTQE'],
 'A10CF04CJZC05F': [],
 'A10EHUTGNC4BGP': [],
 'A10IFYN5U6X20R': [],
 'A10JAUM70F9M8U': [],
 'A10RJEQN64ATXU': [],
 'A10W1JE0YVGXTH': ['A103EZCS9H8WW1',
  'A10EHUTGNC4BGP',
  'A10IFYN5U6X20R',
  'A1YEN91N15OACZ'],
 'A10ZN08SMLZL1Y': [],
 'A117ST5ZLY9YNK': [],
 'A118FLR1Q37TGF': [],
 'A118XYLK4XA1F8': [],
 'A11AT9PG4CZ09P': [],
 'A11LJJL1SOH6W0': [],
 'A11N6Y2ELG1JJN': ['A103EZCS9H8WW1',
  'A10EHUTGNC4BGP',
  'A374D06F5GMJ2R',
  'A39XPX6K05FLLW'],
 'A11NFERETUWA8Z': ['A2CJGVAHVZGJIP', 'A3TF6OME7HCF5F', 'A948FPTGYXYBO'],
 'A11NKS291CE5VK': [],
 'A11QUNPSCNHY62': ['A1O2F0ZB6JVY0Y', 'A21PU11G08TUTL'],
 'A11S4NOJZGS28Q': [],
 'A11UPZ6LI1UJZY': [],
 'A11UZ02KXZ6K98': [],
 'A11V8XN8SUHP

In [70]:
users_products

{'A3SGXH7AUHU8GW': [['B001E4KFG0', 5]],
 'A1D87F6ZCVE5NK': [['B00813GRG4', 1]],
 'ABXLMWJIXXAIN': [['B000LQOCH0', 4]],
 'A395BORC6FGVXV': [['B000UA0QIQ', 2]],
 'A1UQRSCLF8GW1T': [['B006K2ZZ7K', 5]],
 'ADT0SRK1MGOEU': [['B006K2ZZ7K', 4]],
 'A1SP2KVKFXXRU1': [['B006K2ZZ7K', 5]],
 'A3JRGQVEQN31IQ': [['B006K2ZZ7K', 5]],
 'A1MZYO9TZK0BBI': [['B000E7L2R4', 5]],
 'A21BT40VZCCYT4': [['B00171APVA', 5]],
 'A3HDKO7OW0QNK4': [['B0001PB9FE', 5]],
 'A2725IB4YY9JEB': [['B0009XLVG0', 5]],
 'A327PCT23YH90': [['B0009XLVG0', 1]],
 'A18ECVX2RJ7HUE': [['B001GVISJM', 4]],
 'A2MUGFV2TDQ47K': [['B001GVISJM', 5], ['B000G6RYNE', 3], ['B001CWZXIY', 5]],
 'A1CZX3CP8IKQIJ': [['B001GVISJM', 5]],
 'A3KLWF6WQ5BNYO': [['B001GVISJM', 2]],
 'AFKW14U97Z6QO': [['B001GVISJM', 5]],
 'A2A9X58G2GTBLP': [['B001GVISJM', 5]],
 'A3IV7CL2C13K2U': [['B001GVISJM', 5]],
 'A1WO0KGLPR5PV6': [['B001GVISJM', 5]],
 'AZOF9E17RGZH8': [['B001GVISJM', 5]],
 'ARYVQL4N737A1': [['B001GVISJM', 5]],
 'AJ613OLZZUG7V': [['B001GVISJM', 5]],
 'A22P2J0

In [72]:
 users_with_rec_prods = []
recommendations = {}
for user_id in nearest_neighbors.keys():
    recomms_priorities = {}
    if user_id not in recommendations:
        recommendations[user_id] = []
    already_bought = []
    for item in users_products[user_id]:
        already_bought.append(item[0])

    for neighbor in nearest_neighbors[user_id]:
        for product in users_products[neighbor]:
            if product[0] not in already_bought: #pokud uzivatel jeste produkt nekoupil
                if not int(product[1]) < 3: #pokud se sousedovi film libil
                    if not product[0] in recomms_priorities:
                        recomms_priorities[product[0]] = 0
                    recomms_priorities[product[0]] += int(product[1])

    sorted_recomms = {k: v for k, v in sorted(recomms_priorities.items(), key=lambda item: item[1])}

    for s in range(10):
        for item in sorted_recomms.items():
            recommendations[user_id].append(item[0])
        if s == len(sorted_recomms.items())-1:
            break
        if user_id not in users_with_rec_prods:
            users_with_rec_prods.append(user_id)

# INPUT
- Po zadání _UserId_ a požadovaného počtu produktů k doporučení budou vráceny produkty seřazené od toho nejrelevantnějšího.

In [74]:
user_id = input('Enter the Id of user you wish to receive recommendations for (for example A2W3ABLWMJ84NS, A2WH53MCZ2WE3, A2WJH1QD0M9X51, A2WVF9ZQ068DN0, A2ZNLPYMZOXLGX) : ')
n = input('Enter number of recommended products: ')

def main(user_id, n):
    n = int(n)
    to_return = []
    for recommendation in recommendations[user_id]:
        if len(to_return) < n:
            to_return.append(recommendation)
    for recommendation in most_popular.ProductId:
        if len(to_return) < n:
            to_return.append(recommendation)
    return to_return

main(user_id, n)

['B007OXJLM4',
 'B007JFMH8M',
 'B003B3OOPA',
 'B002QWP8H0',
 'B002QWP89S',
 'B002QWHJOU',
 'B0026RQTGE',
 'B001EO5Q64',
 'B000NMJWZO',
 'B007M83302']

In [ ]:
B007OXJLM4

In [75]:
product = pd.read_sql_query("""
select *
from Reviews
where ProductId in ('B007OXJLM4')
""", database)

product

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,2875,B007OXJLM4,A26NFIQ7KWI8Y7,kt rose,2,2,4,1329696000,another great Green Mountain coffee,I found this coffee flavorful and bold. I did...
1,2876,B007OXJLM4,A22SY5TKNKUIEC,Larry,1,1,5,1311638400,greatest coffee,Is so good. Lots of flavor and great coffee ta...
2,2877,B007OXJLM4,A82K4W80VBMKT,bobocaj,0,0,5,1345680000,great deal!,I love how fast my cups arrived. I will always...
3,2878,B007OXJLM4,A12Q6GZEMQOB8B,HBW,0,0,4,1344988800,Best in show for K-cup French Roast,We have several of these machines at work and ...
4,2879,B007OXJLM4,A38QMGAGI3S9CI,Mark van Brenk,0,0,5,1342483200,"Great coffee, great price",I've tried a handful of other brands of French...
5,2880,B007OXJLM4,AL3U2XT77FL1F,Daleen Haifley,0,0,4,1342483200,One of my favorite coffees,"I have had Green Mountain coffee before, but h..."
6,2881,B007OXJLM4,A1JLU3B870VZRC,Donna,0,0,4,1339804800,Great cup of Coffee,"This coffee has a nice bold flavor, not too st..."
7,2882,B007OXJLM4,A2Z06VH8B20ZHF,paula aparis,0,0,5,1327104000,excellent choice,one of the best choices so far. If you like b...
8,2883,B007OXJLM4,A1CODS7MVV1HS0,mary B. floyd,0,0,4,1326412800,Green Mountain K Cups,I prefer stong coffee but my husband does not ...
9,2884,B007OXJLM4,A1VGXHHR08G044,FlyingPolarBear,0,0,4,1323216000,Green Mountain compared with Tully's French Roast,Out of curiosity I compared with side-by-side ...


In [76]:
typek = pd.read_sql_query("""
select *
from Reviews
where UserId in ('A2ZNLPYMZOXLGX')
""", database)

typek

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,374,B002O3VHXU,A2ZNLPYMZOXLGX,"Michael Kelly ""kelly""",0,0,5,1299024000,Nantucket blend k-cups,One of Green Mountains best blends of coffee i...
1,100282,B00954NYVY,A2ZNLPYMZOXLGX,"Michael Kelly ""kelly""",1,2,5,1299024000,Double Black Diamond K- cup,This is a great blend when you need that extra...
2,337436,B006N3I3I4,A2ZNLPYMZOXLGX,"Michael Kelly ""kelly""",1,2,5,1299024000,Double Black Diamond K- cup,This is a great blend when you need that extra...
3,344884,B003S1TQ4O,A2ZNLPYMZOXLGX,"Michael Kelly ""kelly""",0,0,2,1299024000,Apple Cider mix?,What I received was half empty cans of Apple t...
4,517573,B00954NY46,A2ZNLPYMZOXLGX,"Michael Kelly ""kelly""",1,2,5,1299024000,Double Black Diamond K- cup,This is a great blend when you need that extra...
5,520514,B007TJGZ18,A2ZNLPYMZOXLGX,"Michael Kelly ""kelly""",1,2,5,1299024000,Double Black Diamond K- cup,This is a great blend when you need that extra...
6,558284,B001KTA03C,A2ZNLPYMZOXLGX,"Michael Kelly ""kelly""",1,2,5,1299024000,Double Black Diamond K- cup,This is a great blend when you need that extra...


In [ ]:
This is a great blend when you need that extra kick to get you going! Excellent flavor with a Bold blend, "a manly mans blend"!